<a href="https://colab.research.google.com/github/xiyanxiyan10/acm/blob/quant/backtrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install backtrader
!pip install tushare
import backtrader as bt
import tushare as ts
import pandas as pd
from datetime import datetime

In [ ]:
# data cleaning
# get date from tushare
ts.set_token('')

pro = ts.pro_api()

df = pro.coin_bar(start_date='20210615', end_date='20210616',
                  freq='5min', exchange='huobi', ts_code='ETH_USDT')

df.rename(columns={'trade_time':'datetime', 'vol':'volume'}, inplace=True)
df['datetime']=df.datetime.apply(lambda x:pd.to_datetime(x))
df['openinterest']=0
df.index=pd.to_datetime(df.datetime)

print(df)


#回测期间
start=datetime(2021, 6, 15)
end=datetime(2021, 6, 16)
# 加载数据
data = bt.feeds.PandasData(dataname=df,fromdate=start,todate=end)

In [ ]:
# policy 
class MacdStrategy(bt.Strategy):
    """
    继承并构建自己的bt策略
    """

    def log(self, txt, dt=None, doprint=False):
        ''' 日志函数，用于统一输出日志格式 '''
        if doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):

        # 初始化相关数据
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # 五日移动平均线
        self.sma5 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=5)
        # 十日移动平均线
        self.sma10 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=10)

    def notify_order(self, order):
        """
        订单状态处理

        Arguments:
            order {object} -- 订单状态
        """
        if order.status in [order.Submitted, order.Accepted]:
            # 如订单已被处理，则不用做任何事情
            return

        # 检查订单是否完成
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            self.bar_executed = len(self)

        # 订单因为缺少资金之类的原因被拒绝执行
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # 订单状态处理完成，设为空
        self.order = None

    def notify_trade(self, trade):
        """
        交易成果
        
        Arguments:
            trade {object} -- 交易状态
        """
        if not trade.isclosed:
            return

        # 显示交易的毛利率和净利润
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm), doprint=True)

    def next(self):
        ''' 下一次执行 '''
        print(self.dataclose[0])
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 是否正在下单，如果是的话不能提交第二次订单
        if self.order:
            return

        # 是否已经买入
        if not self.position:
            # 还没买，如果 MA5 > MA10 说明涨势，买入
            if self.sma5[0] > self.sma10[0]:
                self.order = self.buy()
        else:
            # 已经买了，如果 MA5 < MA10 ，说明跌势，卖出
            if self.sma5[0] < self.sma10[0]:
                self.order = self.sell()

    def stop(self):
        self.log(u'(金叉死叉有用吗) Ending Value %.2f' %
                 (self.broker.getvalue()), doprint=True)

In [ ]:
cerebro = bt.Cerebro()
cerebro.adddata(data=data)
cerebro.addstrategy(MacdStrategy)
startcash=1000
cerebro.broker.setcash(startcash)
cerebro.broker.setcommission(commission=0.002)

In [ ]:
d1=start.strftime('%Y%m%d')
d2=end.strftime('%Y%m%d')
print(f'初始资金: {startcash}\n回测期间：{d1}:{d2}')
#运行回测系统

cerebro.run()
cerebro.plot()
#portvalue = cerebro.broker.getvalue()
#pnl = portvalue - startcash
#打印结果
#print(f'总资金: {round(portvalue,2)}')